<a href="https://colab.research.google.com/github/A1ienSword/Pattern-recognition-labs/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_13_%D0%9A%D0%BE%D1%81%D1%82%D0%B8%D1%86%D1%8B%D0%BD_%D0%92%D0%92_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
import sys
from tqdm import tqdm

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, img_size=64, num_classes=37):
        """Простая CNN для классификации изображений.

        Параметры:
            img_size (int): размер входного изображения (по умолчанию 64x64)
            num_classes (int): количество классов для классификации (37 = 10 цифр + 26 букв + 1)
        """
        super(SimpleCNN, self).__init__()
        self.img_size = img_size

        # Первый сверточный слой: 1 входной канал, 16 выходных, ядро 5x5
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2)

        # Пулинг для уменьшения размерности в 2 раза
        self.pool = nn.MaxPool2d(2, 2)

        # Второй сверточный слой: 16 входных каналов, 32 выходных
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)

        # Функция активации ReLU (вынесена в атрибут для переиспользования)
        self.relu = nn.ReLU()

        # Динамическая инициализация полносвязных слоев
        self._init_fc_layers()

        # Финальный классификационный слой
        self.fc2 = nn.Linear(128, num_classes)

    def _init_fc_layers(self):
        """Автоматический расчет входных нейронов для полносвязного слоя"""
        # Генерация тестового тензора для расчета размерности
        test_tensor = torch.randn(1, 1, self.img_size, self.img_size)

        with torch.no_grad():
            # Проход через всю сеть для расчета размеров
            x = self.pool(self.relu(self.conv1(test_tensor)))
            x = self.pool(self.relu(self.conv2(x)))

            # "Выпрямление" тензора для полносвязного слоя
            in_features = x.view(-1).shape[0]

        # Инициализация первого полносвязного слоя
        self.fc1 = nn.Linear(in_features, 128)

    def forward(self, x):
        """Прямой проход через сеть"""
        x = self.pool(self.relu(self.conv1(x)))  # Конволюция + активация + пулинг
        x = self.pool(self.relu(self.conv2(x)))  # Второй сверточный блок

        x = x.view(x.size(0), -1)  # Выпрямление в вектор
        x = self.relu(self.fc1(x))  # Полносвязный слой с активацией
        x = self.fc2(x)             # Финальная классификация
        return x

In [ ]:
def train(data_dir='processed_dataset/train',
          model_path='model.pth',
          img_size=64,
          num_classes=37,
          epochs=3,
          batch_size=64,
          learning_rate=0.001):
    """Функция обучения модели

    Параметры:
        data_dir (str): путь к обучающим данным
        model_path (str): путь для сохранения модели
        img_size (int): размер входных изображений
        num_classes (int): количество классов
        epochs (int): количество эпох обучения
        batch_size (int): размер батча
        learning_rate (float): скорость обучения
    """
    # Преобразования для входных изображений
    transform = transforms.Compose([
        transforms.Grayscale(),              # Конвертация в градации серого
        transforms.Resize((img_size, img_size)),  # Изменение размера
        transforms.ToTensor(),               # Конвертация в тензор
        transforms.Normalize((0.5,), (0.5,)) # Нормализация в диапазон [-1, 1]
    ])

    # Загрузка датасета и создание DataLoader
    train_dataset = datasets.ImageFolder(data_dir, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

    # Определение устройства для вычислений (GPU/CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SimpleCNN(img_size=img_size, num_classes=num_classes).to(device)

    # Функция потерь и оптимизатор
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    best_accuracy = 0.0
    progress_bar = tqdm(range(epochs), desc="Обучение модели")

    for epoch in progress_bar:
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for inputs, labels in tqdm(train_loader, desc=f"Эпоха {epoch+1}", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)

            # Обнуление градиентов
            optimizer.zero_grad()

            # Прямой проход
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Обратное распространение
            loss.backward()
            optimizer.step()

            # Статистика
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Вывод статистики
        epoch_accuracy = 100 * correct / total
        progress_bar.set_postfix({
            'loss': f"{total_loss / len(train_loader):.4f}",
            'accuracy': f"{epoch_accuracy:.2f}%"
        })

        # Сохранение лучшей модели
        if epoch_accuracy > best_accuracy:
            best_accuracy = epoch_accuracy
            torch.save({
                'model_state': model.state_dict(),
                'img_size': img_size,
                'num_classes': num_classes
            }, model_path)

    print(f"\nОбучение завершено. Лучшая точность: {best_accuracy:.2f}%")

In [ ]:
def recognize(image_path, model_path='model.pth'):
    """Функция распознавания изображения

    Параметры:
        image_path (str): путь к изображению для классификации
        model_path (str): путь к сохраненной модели
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Загрузка сохраненной модели
    checkpoint = torch.load(model_path, map_location=device)
    img_size = checkpoint['img_size']
    num_classes = checkpoint['num_classes']

    # Инициализация модели
    model = SimpleCNN(img_size=img_size, num_classes=num_classes).to(device)
    model.load_state_dict(checkpoint['model_state'])
    model.eval()  # Режим оценки

    # Преобразования для входного изображения
    transform = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    try:
        # Обработка изображения
        image = Image.open(image_path)
        input_tensor = transform(image).unsqueeze(0).to(device)

        # Предсказание
        with torch.no_grad():
            output = model(input_tensor)
            predicted_class = torch.argmax(output, dim=1).item()

        # Маппинг классов (0-9: цифры, 10-35: A-Z)
        classes = [str(i) for i in range(10)] + [chr(i) for i in range(65, 91)]
        print(f"Предсказанный символ: {classes[predicted_class]}")

    except Exception as e:
        print(f"Ошибка: {str(e)}")

In [ ]:
def test_accuracy(test_dir='processed_dataset/test', model_path='model.pth'):
    """Тестирование точности модели

    Параметры:
        test_dir (str): путь к тестовым данным
        model_path (str): путь к сохраненной модели
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Загрузка модели
    checkpoint = torch.load(model_path, map_location=device)
    img_size = checkpoint['img_size']
    num_classes = checkpoint['num_classes']

    model = SimpleCNN(img_size=img_size, num_classes=num_classes).to(device)
    model.load_state_dict(checkpoint['model_state'])
    model.eval()

    # Преобразования для тестовых данных
    transform = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    # Загрузка тестового датасета
    test_dataset = datasets.ImageFolder(test_dir, transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    progress_bar = tqdm(test_loader, desc="Обработка тестовых данных")

    with torch.no_grad():
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predicted = torch.argmax(outputs, dim=1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            progress_bar.set_postfix({'Текущая точность': f"{100*correct/total:.2f}%"})

    print(f"\nОбщая точность на тестовой выборке: {100*correct/total:.2f}%")

In [ ]:
train()

Обучение модели: 100%|██████████| 3/3 [00:28<00:00,  9.58s/it, loss=0.8411, accuracy=77.84%]


Обучение завершено. Лучшая точность: 77.84%


In [ ]:
test_accuracy()

Обработка тестовых данных: 100%|██████████| 18/18 [00:01<00:00, 12.98it/s, Текущая точность=69.39%]


Общая точность на тестовой выборке: 69.39%


In [ ]:
!unzip processed_dataset.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: processed_dataset/train/X/0017.png  
  inflating: processed_dataset/train/X/0057.png  
  inflating: processed_dataset/train/X/0001.png  
  inflating: processed_dataset/train/X/0051.png  
 extracting: processed_dataset/train/X/0003.png  
  inflating: processed_dataset/train/X/0004.png  
  inflating: processed_dataset/train/X/0049.png  
  inflating: processed_dataset/train/X/0010.png  
  inflating: processed_dataset/train/X/0007.png  
  inflating: processed_dataset/train/X/0023.png  
  inflating: processed_dataset/train/X/0018.png  
 extracting: processed_dataset/train/X/0009.png  
  inflating: processed_dataset/train/X/0013.png  
  inflating: processed_dataset/train/X/0058.png  
  inflating: processed_dataset/train/X/0016.png  
  inflating: processed_dataset/train/X/0047.png  
  inflating: processed_dataset/train/X/0045.png  
  inflating: processed_dataset/train/X/0054.png  
  inflating: processed_dataset/t